In [1]:
import requests
import pandas as pd
import json
from PIL import Image
from pandas.io.json import json_normalize

In [2]:
response=requests.get('https://restcountries.eu/rest/v2/all')

In [3]:
status=response.status_code
status

200

In [4]:
d=response.json()

In [5]:
df=json_normalize(d)
df.shape[0]

250

In [6]:
df.rename(columns={'name':'country','alpha3Code':'alphaCode'},inplace=True)
df.drop(['topLevelDomain','alpha2Code','altSpellings','callingCodes','numericCode','translations.de', 'translations.es',
       'translations.fr', 'translations.ja', 'translations.it',
       'translations.br', 'translations.pt', 'translations.nl',
       'translations.hr', 'translations.fa'],axis=1,inplace=True)

In [7]:
df.head()

,country,alphaCode,capital,region,subregion,population,latlng,demonym,area,gini,timezones,borders,nativeName,currencies,languages,flag,regionalBlocs,cioc
0,Afghanistan,AFG,Kabul,Asia,Southern Asia,27657145,"[33.0, 65.0]",Afghan,652230.0,27.8,[UTC+04:30],"[IRN, PAK, TKM, UZB, TJK, CHN]",افغانستان,"[{'code': 'AFN', 'name': 'Afghan afghani', 'sy...","[{'iso639_1': 'ps', 'iso639_2': 'pus', 'name':...",https://restcountries.eu/data/afg.svg,"[{'acronym': 'SAARC', 'name': 'South Asian Ass...",AFG
1,Åland Islands,ALA,Mariehamn,Europe,Northern Europe,28875,"[60.116667, 19.9]",Ålandish,1580.0,NaN,[UTC+02:00],[],Åland,"[{'code': 'EUR', 'name': 'Euro', 'symbol': '€'}]","[{'iso639_1': 'sv', 'iso639_2': 'swe', 'name':...",https://restcountries.eu/data/ala.svg,"[{'acronym': 'EU', 'name': 'European Union', '...",
2,Albania,ALB,Tirana,Europe,Southern Europe,2886026,"[41.0, 20.0]",Albanian,28748.0,34.5,[UTC+01:00],"[MNE, GRC, MKD, KOS]",Shqipëria,"[{'code': 'ALL', 'name': 'Albanian lek', 'symb...","[{'iso639_1': 'sq', 'iso639_2': 'sqi', 'name':...",https://restcountries.eu/data/alb.svg,"[{'acronym': 'CEFTA', 'name': 'Central Europea...",ALB
3,Algeria,DZA,Algiers,Africa,Northern Africa,40400000,"[28.0, 3.0]",Algerian,2381741.0,35.3,[UTC+01:00],"[TUN, LBY, NER, ESH, MRT, MLI, MAR]",الجزائر,"[{'code': 'DZD', 'name': 'Algerian dinar', 'sy...","[{'iso639_1': 'ar', 'iso639_2': 'ara', 'name':...",https://restcountries.eu/data/dza.svg,"[{'acronym': 'AU', 'name': 'African Union', 'o...",ALG
4,American Samoa,ASM,Pago Pago,Oceania,Polynesia,57100,"[-14.33333333, -170.0]",American Samoan,199.0,NaN,[UTC-11:00],[],American Samoa,"[{'code': 'USD', 'name': 'United State Dollar'...","[{'iso639_1': 'en', 'iso639_2': 'eng', 'name':...",https://restcountries.eu/data/asm.svg,[],ASA


In [8]:
df.columns

Index(['country', 'alphaCode', 'capital', 'region', 'subregion', 'population',
       'latlng', 'demonym', 'area', 'gini', 'timezones', 'borders',
       'nativeName', 'currencies', 'languages', 'flag', 'regionalBlocs',
       'cioc'],
      dtype='object')

In [9]:
#Change area and gini into float type
df['area']=df['area'].astype(float)
df['gini']=df['gini'].astype(float)

In [10]:
df.dtypes

country           object
alphaCode         object
capital           object
region            object
subregion         object
population         int64
latlng            object
demonym           object
area             float64
gini             float64
timezones         object
borders           object
nativeName        object
currencies        object
languages         object
flag              object
regionalBlocs     object
cioc              object
dtype: object

In [11]:
#Function to extract and unlist languages column
def languages(col):
    lang=[]
    for i in range(0,len(col)):
        tab=pd.Series(col[i])
        tab=json_normalize(tab)
        name=tab['name'].tolist()
        name=','.join(name)
        lang.append(name)
    return lang

a=df['languages']
b=languages(a)
df['languages']=b

In [12]:
#Function to extract and unlist currencies column
def currencies(col):
    curr=[]
    for i in range(0,len(col)):
        tab=pd.Series(col[i])
        tab=json_normalize(tab)
        name=tab['name'].to_string()
        name=name.replace('0','').replace('\n1',',')
        curr.append(name)
    return curr

c=df['currencies']
d=currencies(c)
df['currencies']=d

In [13]:
#Function to extract and unlist regionalBlocs column
#Containing [] list of values
def blocs(col):
    bloc=[]   
    for i in range(0,len(col)):
        if len(col[i]) > 0:
            tab=pd.Series(col[i])
            tab=json_normalize(tab)
            name=tab['name'].tolist()
            name=''.join(name)
            bloc.append(name)
        else:
            bloc.append('None')
    return bloc


r=df['regionalBlocs']
t=blocs(r)
df['regionalBlocs']=t

In [14]:
#Unlist columns function
def unlist(col):
    ls=[]
    for i in range(0,250):
        a=col[i]
        a=','.join(a)
        ls.append(a)
    return ls

#Unlist borders column
lis=df['borders']
unlis=unlist(lis)
df['borders']=unlis

In [15]:
#Creat function to fill empty list columns with value None
def fill_empty_list(col):
    fill=[]
    for i in col:
        if len(i) == 0:
            fill.append('None')
        else:
            fill.append(i)
    return fill

#Fill [] empty list values in borders column
bor=df['borders']
bor1=fill_empty_list(bor)
df['borders']=bor1

In [16]:
#Fill [] empty list values in latlng column
latlon=df['latlng']
latlon1=fill_empty_list(latlon)
df['latlng']=latlon1

In [17]:
#There are 3 values presented as Nan in cioc column
#Besides there are some columns shown with empty values not NaN
df['cioc'].fillna('Unknown',inplace=True)

In [18]:
#There are some columns with empty string values which are not shown as NaN
#Fill all those with Unknown string
def fill_empty(col):
    new=[]
    for i in col:
        if i == '':
            new.append('Unknown')
        else:
            new.append(i)
    return new

df['subregion']=fill_empty(df['subregion'])
df['region']=fill_empty(df['region'])
df['demonym']=fill_empty(df['demonym'])
df['cioc']=fill_empty(df['cioc'])
df['capital']=fill_empty(df['capital'])

In [19]:
#Keep NaN values in area and gini columns due to no related information to fill in  
df.isnull().sum()

country           0
alphaCode         0
capital           0
region            0
subregion         0
population        0
latlng            0
demonym           0
area             10
gini             97
timezones         0
borders           0
nativeName        0
currencies        0
languages         0
flag              0
regionalBlocs     0
cioc              0
dtype: int64

In [35]:
df['popDensity']=round(df['population']/df['area'],2)
df.head()

,country,alphaCode,capital,region,subregion,population,latlng,demonym,area,gini,timezones,borders,nativeName,currencies,languages,flag,regionalBlocs,cioc,popDensity
0,Afghanistan,AFG,Kabul,Asia,Southern Asia,27657145,"[33.0, 65.0]",Afghan,652230.0,27.8,[UTC+04:30],"IRN,PAK,TKM,UZB,TJK,CHN",افغانستان,Afghan afghani,"Pashto,Uzbek,Turkmen",https://restcountries.eu/data/afg.svg,South Asian Association for Regional Cooperation,AFG,42.40
1,Åland Islands,ALA,Mariehamn,Europe,Northern Europe,28875,"[60.116667, 19.9]",Ålandish,1580.0,NaN,[UTC+02:00],None,Åland,Euro,Swedish,https://restcountries.eu/data/ala.svg,European Union,Unknown,18.28
2,Albania,ALB,Tirana,Europe,Southern Europe,2886026,"[41.0, 20.0]",Albanian,28748.0,34.5,[UTC+01:00],"MNE,GRC,MKD,KOS",Shqipëria,Albanian lek,Albanian,https://restcountries.eu/data/alb.svg,Central European Free Trade Agreement,ALB,100.39
3,Algeria,DZA,Algiers,Africa,Northern Africa,40400000,"[28.0, 3.0]",Algerian,2381741.0,35.3,[UTC+01:00],"TUN,LBY,NER,ESH,MRT,MLI,MAR",الجزائر,Algerian dinar,Arabic,https://restcountries.eu/data/dza.svg,African UnionArab League,ALG,16.96
4,American Samoa,ASM,Pago Pago,Oceania,Polynesia,57100,"[-14.33333333, -170.0]",American Samoan,199.0,NaN,[UTC-11:00],None,American Samoa,United State Dollar,"English,Samoan",https://restcountries.eu/data/asm.svg,None,ASA,286.93


In [21]:
dff=df.copy()

In [22]:
#Check each region has how many countries
dff['region'].value_counts()

Africa      60
Americas    57
Europe      53
Asia        50
Oceania     27
Unknown      2
Polar        1
Name: region, dtype: int64

### Analyzing asia countries

In [23]:
asia=dff[dff['region']=='Asia'].reset_index(drop=True)
asia

,country,alphaCode,capital,region,subregion,population,latlng,demonym,area,gini,timezones,borders,nativeName,currencies,languages,flag,regionalBlocs,cioc
0,Afghanistan,AFG,Kabul,Asia,Southern Asia,27657145,"[33.0, 65.0]",Afghan,652230.0,27.8,[UTC+04:30],"IRN,PAK,TKM,UZB,TJK,CHN",افغانستان,Afghan afghani,"Pashto,Uzbek,Turkmen",https://restcountries.eu/data/afg.svg,South Asian Association for Regional Cooperation,AFG
1,Armenia,ARM,Yerevan,Asia,Western Asia,2994400,"[40.0, 45.0]",Armenian,29743.0,30.9,[UTC+04:00],"AZE,GEO,IRN,TUR",Հայաստան,Armenian dram,"Armenian,Russian",https://restcountries.eu/data/arm.svg,Eurasian Economic Union,ARM
2,Azerbaijan,AZE,Baku,Asia,Western Asia,9730500,"[40.5, 47.5]",Azerbaijani,86600.0,33.7,[UTC+04:00],"ARM,GEO,IRN,RUS,TUR",Azərbaycan,Azerbaijani manat,Azerbaijani,https://restcountries.eu/data/aze.svg,None,AZE
3,Bahrain,BHR,Manama,Asia,Western Asia,1404900,"[26.0, 50.55]",Bahraini,765.0,NaN,[UTC+03:00],None,‏البحرين,Bahraini dinar,Arabic,https://restcountries.eu/data/bhr.svg,Arab League,BRN
4,Bangladesh,BGD,Dhaka,Asia,Southern Asia,161006790,"[24.0, 90.0]",Bangladeshi,147570.0,32.1,[UTC+06:00],"MMR,IND",Bangladesh,Bangladeshi taka,Bengali,https://restcountries.eu/data/bgd.svg,South Asian Association for Regional Cooperation,BAN
5,Bhutan,BTN,Thimphu,Asia,Southern Asia,775620,"[27.5, 90.5]",Bhutanese,38394.0,38.1,[UTC+06:00],"CHN,IND",ʼbrug-yul,"Bhutanese ngultrum, Indian rupee",Dzongkha,https://restcountries.eu/data/btn.svg,South Asian Association for Regional Cooperation,BHU
6,Brunei Darussalam,BRN,Bandar Seri Begawan,Asia,South-Eastern Asia,411900,"[4.5, 114.66666666]",Bruneian,5765.0,NaN,[UTC+08:00],MYS,Negara Brunei Darussalam,"Brunei dollar, Singapore dollar",Malay,https://restcountries.eu/data/brn.svg,Association of Southeast Asian Nations,BRU
7,Cambodia,KHM,Phnom Penh,Asia,South-Eastern Asia,15626444,"[13.0, 105.0]",Cambodian,181035.0,37.9,[UTC+07:00],"LAO,THA,VNM",Kâmpŭchéa,"Cambodian riel, United States dollar",Khmer,https://restcountries.eu/data/khm.svg,Association of Southeast Asian Nations,CAM
8,China,CHN,Beijing,Asia,Eastern Asia,1377422166,"[35.0, 105.0]",Chinese,9640011.0,47.0,[UTC+08:00],"AFG,BTN,MMR,HKG,IND,KAZ,PRK,KGZ,LAO,MAC,MNG,PA...",中国,Chinese yuan,Chinese,https://restcountries.eu/data/chn.svg,None,CHN
9,Georgia,GEO,Tbilisi,Asia,Western Asia,3720400,"[42.0, 43.5]",Georgian,69700.0,41.3,[UTC-05:00],"ARM,AZE,RUS,TUR",საქართველო,Georgian Lari,Georgian,https://restcountries.eu/data/geo.svg,None,GEO


In [24]:
asia['subregion'].value_counts()

Western Asia          17
South-Eastern Asia    11
Southern Asia          9
Eastern Asia           8
Central Asia           5
Name: subregion, dtype: int64

In [25]:
asia['subregion'].unique()

array(['Southern Asia', 'Western Asia', 'South-Eastern Asia',
       'Eastern Asia', 'Central Asia'], dtype=object)

In [26]:
sea=asia[asia['subregion']=='South-Eastern Asia']
sea

,country,alphaCode,capital,region,subregion,population,latlng,demonym,area,gini,timezones,borders,nativeName,currencies,languages,flag,regionalBlocs,cioc
6,Brunei Darussalam,BRN,Bandar Seri Begawan,Asia,South-Eastern Asia,411900,"[4.5, 114.66666666]",Bruneian,5765.0,NaN,[UTC+08:00],MYS,Negara Brunei Darussalam,"Brunei dollar, Singapore dollar",Malay,https://restcountries.eu/data/brn.svg,Association of Southeast Asian Nations,BRU
7,Cambodia,KHM,Phnom Penh,Asia,South-Eastern Asia,15626444,"[13.0, 105.0]",Cambodian,181035.0,37.9,[UTC+07:00],"LAO,THA,VNM",Kâmpŭchéa,"Cambodian riel, United States dollar",Khmer,https://restcountries.eu/data/khm.svg,Association of Southeast Asian Nations,CAM
12,Indonesia,IDN,Jakarta,Asia,South-Eastern Asia,258705000,"[-5.0, 120.0]",Indonesian,1904569.0,34.0,"[UTC+07:00, UTC+08:00, UTC+09:00]","TLS,MYS,PNG",Indonesia,Indonesian rupiah,Indonesian,https://restcountries.eu/data/idn.svg,Association of Southeast Asian Nations,INA
21,Lao People's Democratic Republic,LAO,Vientiane,Asia,South-Eastern Asia,6492400,"[18.0, 105.0]",Laotian,236800.0,36.7,[UTC+07:00],"MMR,KHM,CHN,THA,VNM",ສປປລາວ,Lao kip,Lao,https://restcountries.eu/data/lao.svg,Association of Southeast Asian Nations,LAO
24,Malaysia,MYS,Kuala Lumpur,Asia,South-Eastern Asia,31405416,"[2.5, 112.5]",Malaysian,330803.0,46.2,[UTC+08:00],"BRN,IDN,THA",Malaysia,Malaysian ringgit,Malaysian,https://restcountries.eu/data/mys.svg,Association of Southeast Asian Nations,MAS
27,Myanmar,MMR,Naypyidaw,Asia,South-Eastern Asia,51419420,"[22.0, 98.0]",Burmese,676578.0,NaN,[UTC+06:30],"BGD,CHN,IND,LAO,THA",Myanma,Burmese kyat,Burmese,https://restcountries.eu/data/mmr.svg,Association of Southeast Asian Nations,MYA
33,Philippines,PHL,Manila,Asia,South-Eastern Asia,103279800,"[13.0, 122.0]",Filipino,342353.0,43.0,[UTC+08:00],None,Pilipinas,Philippine peso,English,https://restcountries.eu/data/phl.svg,Association of Southeast Asian Nations,PHI
36,Singapore,SGP,Singapore,Asia,South-Eastern Asia,5535000,"[1.36666666, 103.8]",Singaporean,710.0,48.1,[UTC+08:00],None,Singapore,"Brunei dollar, Singapore dollar","English,Malay,Tamil,Chinese",https://restcountries.eu/data/sgp.svg,Association of Southeast Asian Nations,SIN
42,Thailand,THA,Bangkok,Asia,South-Eastern Asia,65327652,"[15.0, 100.0]",Thai,513120.0,40.0,[UTC+07:00],"MMR,KHM,LAO,MYS",ประเทศไทย,Thai baht,Thai,https://restcountries.eu/data/tha.svg,Association of Southeast Asian Nations,THA
43,Timor-Leste,TLS,Dili,Asia,South-Eastern Asia,1167242,"[-8.83333333, 125.91666666]",East Timorese,14874.0,31.9,[UTC+09:00],IDN,Timor-Leste,"United States dollar, None",Portuguese,https://restcountries.eu/data/tls.svg,None,TLS


In [27]:
#The most populous country in South-Eastern Asia
sea[sea['population']==sea['population'].max()]

,country,alphaCode,capital,region,subregion,population,latlng,demonym,area,gini,timezones,borders,nativeName,currencies,languages,flag,regionalBlocs,cioc
12,Indonesia,IDN,Jakarta,Asia,South-Eastern Asia,258705000,"[-5.0, 120.0]",Indonesian,1904569.0,34.0,"[UTC+07:00, UTC+08:00, UTC+09:00]","TLS,MYS,PNG",Indonesia,Indonesian rupiah,Indonesian,https://restcountries.eu/data/idn.svg,Association of Southeast Asian Nations,INA


In [28]:
#The least populous country
sea[sea['population']==sea['population'].min()]

,country,alphaCode,capital,region,subregion,population,latlng,demonym,area,gini,timezones,borders,nativeName,currencies,languages,flag,regionalBlocs,cioc
6,Brunei Darussalam,BRN,Bandar Seri Begawan,Asia,South-Eastern Asia,411900,"[4.5, 114.66666666]",Bruneian,5765.0,NaN,[UTC+08:00],MYS,Negara Brunei Darussalam,"Brunei dollar, Singapore dollar",Malay,https://restcountries.eu/data/brn.svg,Association of Southeast Asian Nations,BRU


In [34]:
dff['density']=round(dff['population']/dff['area'],2)
dff.head()

,country,alphaCode,capital,region,subregion,population,latlng,demonym,area,gini,timezones,borders,nativeName,currencies,languages,flag,regionalBlocs,cioc,density
0,Afghanistan,AFG,Kabul,Asia,Southern Asia,27657145,"[33.0, 65.0]",Afghan,652230.0,27.8,[UTC+04:30],"IRN,PAK,TKM,UZB,TJK,CHN",افغانستان,Afghan afghani,"Pashto,Uzbek,Turkmen",https://restcountries.eu/data/afg.svg,South Asian Association for Regional Cooperation,AFG,42.40
1,Åland Islands,ALA,Mariehamn,Europe,Northern Europe,28875,"[60.116667, 19.9]",Ålandish,1580.0,NaN,[UTC+02:00],None,Åland,Euro,Swedish,https://restcountries.eu/data/ala.svg,European Union,Unknown,18.28
2,Albania,ALB,Tirana,Europe,Southern Europe,2886026,"[41.0, 20.0]",Albanian,28748.0,34.5,[UTC+01:00],"MNE,GRC,MKD,KOS",Shqipëria,Albanian lek,Albanian,https://restcountries.eu/data/alb.svg,Central European Free Trade Agreement,ALB,100.39
3,Algeria,DZA,Algiers,Africa,Northern Africa,40400000,"[28.0, 3.0]",Algerian,2381741.0,35.3,[UTC+01:00],"TUN,LBY,NER,ESH,MRT,MLI,MAR",الجزائر,Algerian dinar,Arabic,https://restcountries.eu/data/dza.svg,African UnionArab League,ALG,16.96
4,American Samoa,ASM,Pago Pago,Oceania,Polynesia,57100,"[-14.33333333, -170.0]",American Samoan,199.0,NaN,[UTC-11:00],None,American Samoa,United State Dollar,"English,Samoan",https://restcountries.eu/data/asm.svg,None,ASA,286.93
